# <center> Testing the purification technique from the paper.
## <center> Use only $\ket{0}_E$ to avoid confusions from before.

In [3]:
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

In [4]:
# Okay I am going to forget about everything else and just focus on implementing the purifcation tensor network structure
from opentn.circuits import get_unitary_adchannel
from opentn.channels import get_krauss_from_unitary
from opentn import up,down, plus, minus
import numpy as np

In [5]:
gamma = 0.5
U = get_unitary_adchannel(gamma=gamma)
print(U)

[[ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j  0.70710678+0.j]
 [ 0.        +0.j  0.        +0.j  0.70710678+0.j -0.70710678+0.j]
 [ 0.        +0.j  1.        +0.j  0.        +0.j  0.        +0.j]]


In [6]:
krauss_list = get_krauss_from_unitary(U)
krauss_list

[array([[1.        +0.j, 0.        +0.j],
        [0.        +0.j, 0.70710678+0.j]]),
 array([[0.        +0.j, 0.70710678+0.j],
        [0.        +0.j, 0.        +0.j]])]

In [30]:
# now we need to create new structure for purifcations in TN
# initial purified state
a,b = plus
psi_pure = np.zeros(shape=(1,2,1,1),dtype=np.complex128) #vL vL up down. Assuming environment is by default zero
psi_pure[0,0,0,0] = a
psi_pure[:,1,:,:] = b
psi_pure

array([[[[0.70710678+0.j]],

        [[0.70710678+0.j]]]])

In [36]:
# new purified state after krauss act on physical system
psi_end_pure = np.zeros(shape=(1,2,2,1),dtype=np.complex128) #vL up down vR. Assuming environment is by default zero
psi_end_pure[0,:,0,0] = krauss_list[0]@psi_pure[0,:,0,0]
psi_end_pure[0,:,1,0] = krauss_list[1]@psi_pure[0,:,0,0]
psi_end_pure

array([[[[0.70710678+0.j],
         [0.5       +0.j]],

        [[0.5       +0.j],
         [0.        +0.j]]]])

In [38]:
# tracing out the environment:
"""
        __|__      
       /  1   \  
    ---|0 P 3|--- 
       \__2__/    
          |      
     
        __|__     
       /  2  \    
    ---|0 P* 3|--- 
       \__1__/    
          |       
"""
rho_A = np.tensordot(psi_end_pure, psi_end_pure.conj(),axes=(2,2)) # vL up (down) vR x vL* up* (down*) vR* ->  vL up vR vL* up* vR*
np.squeeze(rho_A)

array([[0.75      +0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.25      +0.j]])